## Project Template (Deepnote)

You can use this notebook as a starting point for your class project and/or the extra exercise for graduate students in Deepnote. It comes set up with the dependencies we use in class, so you don't have to install these yourself.

**Notes:**
- To use this in your own workspace, click “Duplicate” for the project in Deepnote.
- Class dependencies, namely `drake` and the `manipulation` package, are already preinstalled in the Deepnote image for this course, as well as a bunch of other dependencies that these packages depend on or that we use in class
- To see the full list of installed dependencies, as well as version numbers etc, see this file: [pyproject.toml](https://github.com/RussTedrake/manipulation/blob/master/pyproject.toml).
  - **Note**: we install all the extra dependencies, including the `dev` dependencies, into the Docker image that this deepnote project runs on.


### (Advanced Users) Installing additional dependencies

Here we show you how to install additional packages into this Deepnote project. We will use the package [gcsopt](https://pypi.org/project/gcsopt/) as an example. To install additional dependencies, you have two choices:
- **Recommended** (automatically installs your dependencies every time you boot the Deepnote machine):
  1. Add the dependency to the list of dependencies in `reqirements.txt` on the left.  
      - **Note**: Make sure to NOT remove the text that is already in the file. You should add your dependencies on new lines AFTER `manipulation=year.month.day`. For our example, the file contents will look like:
          ```
          --extra-index-url https://drake-packages.csail.mit.edu/whl/nightly
          manipulation==2025.9.23
          gcsopt
          ```
  2. Restart the Deepnote machine (or run the `Init` notebook)
- **Alternatively:** Run `!pip install gcsopt` within a cell in the Deepnote notebook. You will have to do this everytime you restart your Deepnote machine.

### (Advanced Users) Updating the `manipulation` version
We continually add new features to the `manipulation` package, which is automatically updated in Deepnote. However, after you first click "Duplicate", you will stop receiving these updates. Follow these steps to update your version to the latest version:
1. Find the latest version number for `manipulation` on the [PyPi package page](https://pypi.org/project/manipulation/#history).
2. Update the version number for `manipulation=year.month.day` in the `requirements.txt` file on the left
3. Restart the Deepnote machine (or run the `Init` notebook)

The changes will persist every time you start your Deepnote machine.

Good luck!

---


## Placeholder code: iiwa Simulation

As a starting point, the code below sets up a simple simulation with a single iiwa, visualized in Meshcat.

### Setup and imports
Let us first get our imports out of the way:

In [ ]:
from pydrake.all import (
    DiagramBuilder,
    Simulator,
    StartMeshcat,
    BasicVector,
    LeafSystem,
    InverseKinematics,
    SnoptSolver,
    RotationMatrix,
    RigidTransform,
    Solve,
    LogVectorOutput,
    PiecewisePolynomial,
    TrajectorySource,
    ImageWriter,
    PixelType
)

from manipulation.station import (
    LoadScenario,
    MakeHardwareStation,
)

from manipulation import running_as_notebook

import matplotlib.pyplot as plt
import numpy as np
import pygame
import math
import os
import h5py
import numpy as np
import collections

ModuleNotFoundError: No module named 'act'

### Meshcat Visualization

As always, let's start Meshcat for our 3D visualization!

In [6]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7001


Click the link above to open Meshcat in your browser!


### Testing Xbox Controller

In [7]:
pygame.init()  # Initialize all Pygame modules
pygame.joystick.init() # Initialize the joystick module

joystick_count = pygame.joystick.get_count()
if joystick_count == 0:
    print("No joysticks found.")
else:
    # Initialize the first detected joystick (assuming one Xbox controller)
    joystick = pygame.joystick.Joystick(0)
    joystick.init()
    print(f"Controller detected: {joystick.get_name()}")
    
# --- Configuration ---
# Dead Zone threshold for stick inputs
DEAD_ZONE = 0.15 
POLLING_RATE_HZ = 60 

AXIS_LEFT_X  = 0
AXIS_LEFT_Z  = 1
AXIS_RIGHT_X = 2
AXIS_RIGHT_Z = 3
AXIS_LEFT_TRIGGER  = 4
AXIS_RIGHT_TRIGGER = 5 
BUTTON_DPAD_LEFT  = 13
BUTTON_DPAD_RIGHT = 14

def filter_stick_input(value):
    """
    Applies a dead zone filter and scales the remaining input linearly.
    """
    if abs(value) < DEAD_ZONE:
        return 0.0
    
    scaled_value = (abs(value) - DEAD_ZONE) / (1.0 - DEAD_ZONE)
    return math.copysign(scaled_value, value)


No joysticks found.


### Setting up the Scenario

In [8]:
scenario_string = """directives:
# add robot
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/urdf/iiwa14_primitive_collision.urdf
    default_joint_positions:
      iiwa_joint_1: [-1.57]
      iiwa_joint_2: [0.1]
      iiwa_joint_3: [0]
      iiwa_joint_4: [-1.2]
      iiwa_joint_5: [0]
      iiwa_joint_6: [1.6]
      iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0

# add gripper
- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}

# add camera mounted to world 
- add_frame:
    name: camera0_origin
    X_PF:
        base_frame: world
        rotation: !Rpy { deg: [270, 0.0, 90.0]}
        translation: [1.2, -0.5, 0.5]
- add_model:
    name: camera0
    file: package://manipulation/camera_box.sdf
- add_weld:
    parent: camera0_origin
    child: camera0::base

# add camera mounted to robot wrist
- add_frame:
    name: camera_wrist
    X_PF:
      base_frame: iiwa::iiwa_link_7
      translation: [-0.05, 0, 0.1]   # 10 cm ahead of wrist
      rotation: !Rpy {deg: [0, 00, -90]}
- add_model:
    name: camera1
    file: package://manipulation/camera_box.sdf
- add_weld:
    parent: iiwa::camera_wrist
    child: camera1::base

# add cabinet
- add_model:
    name: cabinet
    file: package://drake_models/manipulation_station/cupboard.sdf
- add_frame:
    name: cabinet_origin
    X_PF:
      base_frame: world
      translation: [0, -1.1, 0.4]    # x, y, z in meters
      rotation: !Rpy { deg: [0, 0, 90]}  # roll, pitch, yaw
- add_weld:
    parent: cabinet_origin
    child: cabinet::cupboard_body

cameras:
  camera0:
    name: camera0
    depth: True
    X_PB:
      base_frame: camera0::base
  camera1:
    name: camera1
    depth: True
    X_PB:
      base_frame: camera1::base

model_drivers:
  iiwa: !IiwaDriver
    control_mode: position_only 
    hand_model_name: wsg
  wsg: !SchunkWsgDriver {}
"""

# scenario = LoadScenario(data=scenario_string)
# station = MakeHardwareStation(scenario, meshcat=meshcat)
# builder = DiagramBuilder()
# builder.AddSystem(station)
# diagram = builder.Build()
# context = diagram.CreateDefaultContext()

# simulator = Simulator(diagram)
# simulator.set_target_realtime_rate(1.0)
# simulator.AdvanceTo(0.0)

# Teleopping

In [9]:
class TeleopController(LeafSystem):
    """
    Buttons for end-effector pose to joint positions via ik
    """
    def __init__(self, meshcat, plant):
        LeafSystem.__init__(self)
        
        # Movement params
        self.STEP_SIZE_M = 0.01  
        self.STEP_SIZE_RAD = 0.087 # ~5 degrees
        
        # --- PYGAME & CONTROLLER INITIALIZATION ---
        try:
            pygame.init()
            pygame.joystick.init()
            if pygame.joystick.get_count() == 0:
                print("ERROR: No joystick/controller detected! Teleop will be disabled.")
                self.controller = None
            else:
                self.controller = pygame.joystick.Joystick(0)
                self.controller.init()
                print(f"Controller Connected: {self.controller.get_name()}")
        except Exception as e:
            print(f"ERROR: Pygame initialization failed: {e}")
            self.controller = None

        
        self.meshcat = meshcat
        self.plant = plant
        
        # getting the necessary frames
        self.plant_context = plant.CreateDefaultContext()
        self.W_frame = plant.world_frame()

        wsg_instance = self.plant.GetModelInstanceByName("wsg")
        wsg_link = self.plant.GetBodyByName("body", wsg_instance)
        self.WG_frame = wsg_link.body_frame()
        
        self.nq = 7 
        
        # Configuration for IK
        self.q_initial = np.array([-1.57, 0.2, 0, -1.2, 0, 1.6, 0])
        self.iiwa = self.plant.GetModelInstanceByName("iiwa")
        first_iiwa_joint = self.plant.GetJointByName("iiwa_joint_1", self.iiwa)

        self.iiwa_q_start_index = first_iiwa_joint.position_start()
        
        # Initialize positions
        plant.SetPositions(self.plant_context, self.iiwa, self.q_initial)
        
        # initialize blank desired states
        self.X_WG_desired = plant.CalcRelativeTransform(
            self.plant_context, self.W_frame, self.WG_frame
        )
        self.R_WG_desired = self.X_WG_desired.rotation()
        self.q_desired = self.q_initial
        self.wsg_position_desired = 0.1

        # Register output ports for IIWA and WSG commands
        self.DeclareVectorOutputPort(
            "iiwa_position_command", 
            BasicVector(self.nq), 
            self.DoCalcIiwaOutput
        )
        self.DeclareVectorOutputPort(
            "wsg_position_command", 
            BasicVector(1), 
            lambda context, output: output.SetFromVector([self.wsg_position_desired])
        )

        meshcat.AddButton("Stop Simulation") 

    def read_controller_inputs(self):
        """
        Reads the current state of the Xbox controller and returns normalized 
        XYZ/RPY values.
        """
        if not self.controller:
            return None, None
            
        pygame.event.pump()
        
        # Read raw stick/trigger inputs
        raw_lx = self.controller.get_axis(AXIS_LEFT_X)
        raw_lz = self.controller.get_axis(AXIS_LEFT_Z)
        raw_rx = self.controller.get_axis(AXIS_RIGHT_X)
        raw_rz = self.controller.get_axis(AXIS_RIGHT_Z)

        raw_lt = self.controller.get_axis(AXIS_LEFT_TRIGGER)
        raw_rt = self.controller.get_axis(AXIS_RIGHT_TRIGGER)
        

        # xyz trnaslation
        Y = filter_stick_input(raw_lx)
        Z = filter_stick_input(raw_lz) 
        X = raw_rt - raw_lt 
        
        btn_gripper_close = self.controller.get_button(0) # A button
        btn_gripper_open  = self.controller.get_button(1) # B button

        # RPY rotation
        btn_left  = self.controller.get_button(BUTTON_DPAD_LEFT)
        btn_right = self.controller.get_button(BUTTON_DPAD_RIGHT)
        Pitch = float(btn_right - btn_left)
        Roll = filter_stick_input(raw_rz) 
        Yaw = filter_stick_input(raw_rx)
        
        translation_vector = np.array([X, Y, Z])
        rotation_vector = np.array([Roll, Pitch, Yaw])
        
        return translation_vector, rotation_vector, btn_gripper_close, btn_gripper_open
    
    def step_control(self):
        """
        checks for update events
        """
    
        if not self.controller:
            return

        translation_input, rotation_input, btn_a, btn_b = self.read_controller_inputs()
        delta_p = translation_input * self.STEP_SIZE_M
        
        moved_p = np.linalg.norm(delta_p) > 1e-4
        if moved_p:
            self.X_WG_desired.set_translation(
                self.X_WG_desired.translation() + delta_p
            )
            
        # roll pitch. yaw
        
        roll_rate, pitch_rate, yaw_rate = rotation_input * self.STEP_SIZE_RAD
        delta_R = RotationMatrix.MakeXRotation(roll_rate) \
                  .multiply(RotationMatrix.MakeYRotation(pitch_rate)) \
                  .multiply(RotationMatrix.MakeZRotation(yaw_rate))
                  
        moved_R = np.linalg.norm(rotation_input) > 1e-4

        if moved_R:
            self.R_WG_desired = delta_R.multiply(self.R_WG_desired)

        if btn_a == 1: 
            self.wsg_position_desired = 0.0
        elif btn_b == 1:
            self.wsg_position_desired = 0.1
            
        # recalculate IK if any change occurred
        if moved_p or moved_R or btn_a == 1 or btn_b == 1:
            self.X_WG_desired = RigidTransform(self.R_WG_desired, self.X_WG_desired.translation())
            self._calculate_ik()
            
    def _calculate_ik(self):
        """
        Finds the joint positions (q_desired) that best match the desired
        end-effector pose (X_WG_desired), using the last valid solution as a seed.
        """
        print("finding IK solution...")
        ik = InverseKinematics(self.plant, self.plant_context)
        
        # define decision variables for only the IIWA joints
        q_iiwa = ik.q()[self.iiwa_q_start_index : self.iiwa_q_start_index + self.nq]
        
        p_WG_W = self.X_WG_desired.translation()
        ik.AddPositionConstraint(
            self.WG_frame, [0, 0, 0], self.W_frame, p_WG_W, p_WG_W
        )
        
        R_WG_W = self.X_WG_desired.rotation()
        ik.AddOrientationConstraint(
            self.WG_frame, R_WG_W, self.W_frame, RotationMatrix(), 0.05
        )
        
        prog = ik.prog()
        q_current = self.q_desired
        
        # cost minimizes distance b/w current and future state
        prog.AddQuadraticErrorCost(
            np.identity(self.nq), 
            q_current, 
            q_iiwa
        )
        
        solver = SnoptSolver()
        
        # set initial guess
        q_full_initial_guess = self.plant.GetPositions(self.plant_context)
        
        result = solver.Solve(prog, initial_guess=q_full_initial_guess)

        if result.is_success():
            # update positions
            q_full_result = result.GetSolution(ik.q())
            q_iiwa_result = q_full_result[self.iiwa_q_start_index : self.iiwa_q_start_index + self.nq]
            
            self.q_desired = q_iiwa_result
            self.plant.SetPositions(self.plant_context, self.iiwa, self.q_desired)
        else:
            print(f"IK Warning: Failed to find joint solution. Check if target is reachable.")
            
    def DoCalcIiwaOutput(self, context, output):
        """
        Outputs the latest calculated joint position command.
        """
        output.SetFromVector(self.q_desired)

In [ ]:
WSG_MIN = 0.0        # fully closed width (meters)
WSG_MAX = 0.1        # fully open width (meters)

def normalize_wsg(width):
    return np.clip((width - WSG_MIN) / (WSG_MAX - WSG_MIN), 0.0, 1.0)

def unnormalize_wsg(norm_width):
    return WSG_MIN + norm_width * (WSG_MAX - WSG_MIN)

# TODO: figure out real velocity limit in simulation by closing/opening grippers
WSG_MAX_SPEED = 0.4  # meters/sec width change

def normalize_wsg_velocity(width_velocity):
    v = width_velocity / WSG_MAX_SPEED
    return np.clip(v, -1.0, 1.0)

Run simulation to collect teleoperation data and save it to files.

In [29]:
target_dt = 0.05  # 20 Hz

scenario = LoadScenario(data=scenario_string)
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder = DiagramBuilder()
builder.AddSystem(station)
plant = station.GetSubsystemByName("plant") 

camera_world = station.GetSubsystemByName("rgbd_sensor_camera0")
camera_wrist = station.GetSubsystemByName("rgbd_sensor_camera1")


# add controller
controller = builder.AddSystem(TeleopController(meshcat, plant))

# Connect controller outputs to station inputs
builder.Connect(
    controller.GetOutputPort("iiwa_position_command"),
    station.GetInputPort("iiwa.position")
)
builder.Connect(
    controller.GetOutputPort("wsg_position_command"),
    station.GetInputPort("wsg.position")
)

diagram = builder.Build()
context = diagram.CreateDefaultContext()
station_context = diagram.GetSubsystemContext(station, context)

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)

print("\n--- Starting Cartesian Teleop Demo ---")
print("Please focus the Meshcat viewer window to enable keyboard input.")


image_data_camera0 = []
image_data_camera1 = []
timestamps = []

qpos_list = []
qvel_list = []
action_list = []  # iiwa + gripper commands


# Run until the user clicks the "Stop Simulation" button in Meshcat
while meshcat.GetButtonClicks('Stop Simulation') == 0:
    controller.step_control()
    
    simulator.AdvanceTo(simulator.get_context().get_time() + target_dt)

    sim_context = simulator.get_context()
    station_context = diagram.GetSubsystemContext(station, sim_context)

    # ---------- get plant state (qpos, qvel) ----------
    plant_context = station.GetSubsystemContext(plant, station_context)
    iiwa_model = plant.GetModelInstanceByName("iiwa")
    wsg_model  = plant.GetModelInstanceByName("wsg")

    # iiwa joints
    q_iiwa = plant.GetPositions(plant_context, iiwa_model)
    v_iiwa = plant.GetVelocities(plant_context, iiwa_model)

    # gripper joints
    q_wsg = plant.GetPositions(plant_context, wsg_model)
    v_wsg = plant.GetVelocities(plant_context, wsg_model)
    wsg_width = normalize_wsg(q_wsg[1] - q_wsg[0])
    wsg_vel = normalize_wsg_velocity(v_wsg[1] - v_wsg[0])
    print(v_wsg[1] - v_wsg[0])

    # concatenate to get exactly (7 + 1,) vectors
    qpos = np.concatenate([q_iiwa, [wsg_width]], axis=0).copy()
    qvel = np.concatenate([v_iiwa, [wsg_vel]], axis=0).copy()

    qpos_list.append(qpos)
    qvel_list.append(qvel)

     # ---------- get current action (controller outputs) ----------
    controller_context = diagram.GetSubsystemContext(controller, sim_context)

    iiwa_cmd = controller.GetOutputPort("iiwa_position_command").Eval(controller_context)
    wsg_cmd = controller.GetOutputPort("wsg_position_command").Eval(controller_context)
    # normalize gripper commands
    wsg_cmd = [normalize_wsg(x) for x in wsg_cmd]

    # flatten to 1D arrays and concatenate
    iiwa_cmd_np = np.asarray(iiwa_cmd).ravel()
    wsg_cmd_np = np.asarray(wsg_cmd).ravel()
    action = np.concatenate([iiwa_cmd_np, wsg_cmd_np], axis=0)
    action_list.append(action)

    # Grab the current images from the output ports
    img0 = station.GetOutputPort("camera0.rgb_image").Eval(station_context)  # ImageRgba8U object
    img1 = station.GetOutputPort("camera1.rgb_image").Eval(station_context)  # ImageRgba8U object

    # Convert to numpy arrays for easier handling
    np_img0 = np.array(np.copy(img0.data)).reshape(img0.height(), img0.width(), -1)  # RGBA
    np_img1 = np.array(np.copy(img1.data)).reshape(img1.height(), img1.width(), -1)
    
    # keep only RGB channels; ACT expects (H, W, 3) uint8 images
    image_data_camera0.append(np_img0[:, :, :3].copy())
    image_data_camera1.append(np_img1[:, :, :3].copy())

    timestamps.append(sim_context.get_time())

print("Simulation stopped by user.")

ERROR: No joystick/controller detected! Teleop will be disabled.

--- Starting Cartesian Teleop Demo ---
Please focus the Meshcat viewer window to enable keyboard input.
0.7553897339098661
-0.03483992777266588
-0.002044353539566491
0.00028500066263665354
-1.3624383762542528e-06
-1.4061054502610476e-06
7.18425267224508e-08
3.9237800587195394e-09
-5.543804149054299e-10
4.2292249512611286e-12
2.2510035511673243e-12
-2.796497712847023e-13
-5.641506791421523e-14
-1.7825896045528374e-14
-6.642726104524485e-15
-2.190738986726526e-15
-1.112193475323712e-15
-3.7198961473583417e-16
-2.891773141699406e-16
1.526409484220639e-16
7.317502680107066e-17
4.5029513304955594e-17
3.7469236661987474e-17
5.1426572985587625e-17
2.241761939648168e-17
5.19216821315037e-18
1.6739110577107085e-17
1.8241045063866066e-18
-1.4313998068710396e-16
3.8055672028423744e-18
5.459240546570199e-19
-3.497074889095122e-18
1.624811585583216e-16
4.0158309642191384e-17
-1.5367520467978858e-17
1.1037205509575261e-16
9.0196042817

In [ ]:
# ---------- convert buffers to numpy arrays ----------
qpos_arr = np.stack(qpos_list, axis=0).astype(np.float32)      # (T, nq)
qvel_arr = np.stack(qvel_list, axis=0).astype(np.float32)      # (T, nq)
action_arr = np.stack(action_list, axis=0).astype(np.float32)  # (T, act_dim)

images0_arr = np.stack(image_data_camera0, axis=0).astype(np.uint8)  # (T, H, W, 3)
images1_arr = np.stack(image_data_camera1, axis=0).astype(np.uint8)  # (T, H, W, 3)

timestamps_arr = np.array(timestamps, dtype=np.float64)

print("qpos: ", qpos_arr, qpos_arr.shape)
print("qvel: ", qvel_arr, qvel_arr.shape)
print("actions: ", action_arr, action_arr.shape)
print(timestamps_arr)

# ---------- write ACT-style HDF5 episode ----------
episode_idx = 0  # you can change this per demonstration
h5_path = os.path.join("teleop_data/sim_open_drawer", f"episode_{episode_idx}.hdf5")

with h5py.File(h5_path, "w") as root:
    # mark this as simulation data (ACT uses this flag)
    root.attrs["sim"] = True

    # actions: shape (T, act_dim)
    root.create_dataset("action", data=action_arr, compression="gzip")

    # observations group
    obs_grp = root.create_group("observations")
    obs_grp.create_dataset("qpos", data=qpos_arr, compression="gzip")
    obs_grp.create_dataset("qvel", data=qvel_arr, compression="gzip")

    # optional: store timestamps (not strictly required by ACT but often useful)
    obs_grp.create_dataset("time", data=timestamps_arr)

    # images subgroup, one dataset per camera
    img_grp = obs_grp.create_group("images")
    img_grp.create_dataset("camera0", data=images0_arr, compression="gzip")
    img_grp.create_dataset("camera1", data=images1_arr, compression="gzip")

print(f"Wrote ACT-style episode to {h5_path}")

qpos:  [[-1.5699646e+00  1.4605041e-01  1.1644751e-04 -1.2041779e+00
   1.3209711e-04  1.6026086e+00  9.3513394e-05  9.3241411e-01]
 [-1.5699916e+00  1.8585697e-01  3.7102491e-05 -1.2013375e+00
   4.2040003e-05  1.6018995e+00  6.6869732e-05  1.0000000e+00]
 [-1.5699985e+00  1.9698377e-01  8.3490895e-06 -1.2003013e+00
   9.4544657e-06  1.6009487e+00  3.3229226e-05  9.9994642e-01]
 [-1.5699997e+00  1.9935815e-01  1.8144767e-06 -1.2000655e+00
   2.0534408e-06  1.6004297e+00  1.5018778e-05  9.9995887e-01]
 [-1.5699999e+00  1.9987340e-01  3.6488697e-07 -1.2000132e+00
   4.1259617e-07  1.6001798e+00  6.2751310e-06  1.0000000e+00]
 [-1.5700001e+00  1.9997790e-01  6.8200080e-08 -1.2000024e+00
   7.7005218e-08  1.6000723e+00  2.5221102e-06  1.0000000e+00]
 [-1.5700001e+00  1.9999884e-01  8.5240108e-09 -1.2000003e+00
   9.5653263e-09  1.6000283e+00  9.8318014e-07  1.0000000e+00]
 [-1.5700001e+00  2.0000303e-01 -3.3897884e-09 -1.1999999e+00
  -3.8815000e-09  1.6000106e+00  3.6919468e-07  1.000000

Replay collected joint and gripper data. (Need to restart kernel, run first three cells to import dependencies and define scenario_string and meshcat, then run this cell.)

In [6]:
joint_data = np.load("downsampled_teleop_joint_data.npz")
joint_t_uniform = joint_data["time"]
joint_q_uniform = joint_data["q"]

gripper_data = np.load("downsampled_teleop_gripper_data.npz")
gripper_t_uniform = gripper_data["time"]
gripper_q_uniform = gripper_data["q"]

scenario = LoadScenario(data=scenario_string)
station = MakeHardwareStation(scenario, meshcat=meshcat)
plant = station.GetSubsystemByName("plant") 

traj_iiwa = PiecewisePolynomial.FirstOrderHold(joint_t_uniform, joint_q_uniform)
traj_wsg = PiecewisePolynomial.FirstOrderHold(gripper_t_uniform, gripper_q_uniform)

builder = DiagramBuilder()
builder.AddSystem(station)  # your station system already created

# Create trajectory sources
iiwa_source = builder.AddSystem(TrajectorySource(traj_iiwa))
wsg_source = builder.AddSystem(TrajectorySource(traj_wsg))

# Connect to the plant
builder.Connect(iiwa_source.get_output_port(0),
                station.GetInputPort("iiwa.position"))
builder.Connect(wsg_source.get_output_port(0),
                station.GetInputPort("wsg.position"))

diagram = builder.Build()

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)

# Run for the duration of the trajectory
simulator.AdvanceTo(joint_t_uniform[-1])

print("Replay finished!")

Replay finished!


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cf4fb7c2-18e7-48b8-93e7-63f0209e538b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>